# CME 241 Assignment 13

## Shaan Patel

### Question 1

In [3]:
from typing import Iterable, Iterator, TypeVar, Mapping, Callable
import rl.markov_decision_process as mp
from rl.approximate_dynamic_programming import NTStateDistribution, QValueFunctionApprox
from rl.policy import Policy, DeterministicPolicy, RandomPolicy, UniformPolicy
from rl.distribution import Categorical
from rl.returns import returns
from collections import defaultdict

In [5]:
S = TypeVar('S')
A = TypeVar('A')

def greedy_policy_from_qvf(
    q: QValueFunctionApprox[S,A],
    actions: Callable[[mp.NonTerminal[S]], Iterable[A]]
) -> DeterministicPolicy[S,A]:
    def optimal_action(s: S) -> A:
        _, a = q.argmax((mp.NonTerminal(s), a) for a in actions(mp.NonTerminal(s)))
        return a
    return DeterministicPolicy(optimal_action)


def epsilon_greedy_policy(
    q: QValueFunctionApprox[S,A],
    mdp: mp.MarkovDecisionProcess[S,A],
    epsilon: float = 0.0
) -> Policy[S,A]:
    def explore(s: S, mdp=mdp) -> Iterable[A]:
        return mdp.actions(mp.NonTerminal(s))
    return RandomPolicy(Categorical(
        {UniformPolicy(explore): epsilon,
        greedy_policy_from_qvf(q, mdp.actions): 1 - epsilon}
    ))


def mc_glie(
    mdp: mp.MarkovDecisionProcess[S,A],
    states: NTStateDistribution[S],
    approx_0: QValueFunctionApprox[S,A],
    gamma: float,
    epsilon_func: Callable[[int], float],
    episode_length_tolerance: float = 1e-6
) -> Iterator[QValueFunctionApprox[S,A]]:

    num_episodes = 0
    q: QValueFunctionApprox[S,A] = approx_0
    p: Policy[S,A] = epsilon_greedy_policy(q,mdp, 1.0)
    yield q

    while True:
        trace: Iterable[mp.TransitionStep[S,A]] = mdp.simulate_actions(states,p)
        num_episodes += 1
        for step in returns(trace, gamma, episode_length_tolerance):
            q = q.update([((step.state, step.action), step.return_)])
        p = epsilon_greedy_policy(q,mdp,
        epsilon_func(num_episodes))
        yield q



In [6]:
def tabular_gpfq(
    q: Mapping[S,Mapping[A, float]],
    actions: Callable[[mp.NonTerminal[S]], Iterable[A]]
) -> DeterministicPolicy[S,A]:
    def optimal_action(s:S) -> A:
        max_act = max(q[s], key = q[s].get)
        return max_act
    return DeterministicPolicy(optimal_action)


def tabular_egp(
    q: Mapping[S, Mapping[A, float]],
    mdp: mp.MarkovDecisionProcess[S,A],
    epsilon: float = 0.0
) -> Policy[S,A]:
    def explore(s: S, mdp=mdp) -> Iterable[A]:
        return mdp.actions(mp.NonTerminal(s))
    return RandomPolicy(Categorical(
        {UniformPolicy(explore): epsilon,
        tabular_gpfq(q, mdp.actions): 1 - epsilon}
    ))

def tabular_mc_glie(
    mdp: mp.MarkovDecisionProcess[S,A],
    states: NTStateDistribution[S],
    approx_0: Mapping[S, Mapping[A, float]],
    gamma: float,
    episode_length_tolerance: float = 1e-6
) -> Iterator[Mapping[S, Mapping[A, float]]]:

    num_episodes = 0
    counts = defaultdict(lambda: 0)
    q: Mapping[S,Mapping[A,float]] = approx_0
    p: Policy[S,A] = tabular_egp(q,mdp,1.0)

    yield q

    while True:
        trace: Iterable[mp.TransitionStep[S,A]] = mdp.simulate_actions(states, p)
        num_episodes += 1
        for step in returns(trace, gamma, episode_length_tolerance):
            counts[(step.state, step.action)] += 1
            q[step.state][step.action] += (1/counts[(step.state,step.action)])*\
                (step.return_ - q[step.state][step.action])
        epsilon = 1/num_episodes
        p = tabular_egp(q,mdp,epsilon)
        yield q



### Question 2

In [8]:
from operator import itemgetter
from typing import Set

In [ ]:
def epsilon_greedy_action(
    q: QValueFunctionApprox[S, A],
    nt_state: mp.NonTerminal[S],
    actions: Set[A],
    epsilon: float
) -> A:
    greedy_action: A = max(
        ((a, q((nt_state, a))) for a in actions),
        key=itemgetter(1)
    )[0]
    return Categorical(
        {a: epsilon / len(actions) +
        (1 - epsilon if a == greedy_action else 0.) for a in actions}
    ).sample()


def glie_sarsa(
    mdp: mp.MarkovDecisionProcess[S, A],
    states: NTStateDistribution[S],
    approx_0: QValueFunctionApprox[S, A],
    gamma: float,
    epsilon_as_func_of_episodes: Callable[[int], float],
    max_episode_length: int
) -> Iterator[QValueFunctionApprox[S, A]]:
    q: QValueFunctionApprox[S, A] = approx_0
    yield q
    num_episodes: int = 0
    while True:
        num_episodes += 1
        epsilon: float = epsilon_as_func_of_episodes(num_episodes)
        state: mp.NonTerminal[S] = states.sample()
        action: A = epsilon_greedy_action(
            q=q,
            nt_state=state,
            actions=set(mdp.actions(state)),
            epsilon=epsilon
        )
        steps: int = 0
        while isinstance(state, mp.NonTerminal) and steps < max_episode_length:
            next_state, reward = mdp.step(state, action).sample()
            if isinstance(next_state, mp.NonTerminal):
                next_action: A = epsilon_greedy_action(
                    q=q,
                    nt_state=next_state,
                    actions=set(mdp.actions(next_state)),
                    epsilon=epsilon
                )
                q = q.update([(
                    (state, action),
                    reward + gamma * q((next_state, next_action))
                )])
                action = next_action
            else:
                q = q.update([((state, action), reward)])
            yield q
            steps += 1
            state = next_state

In [9]:
def tabular_ega(
    q: Mapping[S, Mapping[A, float]],
    states: mp.NonTerminal[S],
    actions: Set[A],
    epsilon: float
) -> A:
    greedy_action: A = max(
        (q[states][a] for a in actions),
        key = q[states].get
    )
    return Categorical(
        {a: epsilon / len(actions) +
        (1 - epsilon if a == greedy_action else 0.) for a in actions}
    ).sample()


def tabular_sarsa(
    mdp: mp.MarkovDecisionProcess[S,A],
    states: NTStateDistribution[S],
    approx_0: Mapping[S, Mapping[A, float]],
    gamma: float,
    max_episode_length: int
) -> Iterator[Mapping[S,Mapping[A,float]]]:

    counts = defaultdict(lambda: 0)
    q: Mapping[S,Mapping[A,float]] = approx_0
    yield q
    num_episodes: int = 0

    while True:
        num_episodes += 1
        epsilon: float = 1/num_episodes
        state: mp.NonTerminal[S] = states.sample()
        action: A = tabular_ega(
            q=q,
            states=state,
            actions=set(mdp.actions(state)),
            epsilon=epsilon
        )
        steps: int = 0
        while isinstance(state,mp.NonTerminal) and steps < max_episode_length:
            counts[(state,action)] += 1
            next_state, reward = mdp.step(state, action).sample()
            if isinstance(next_state, mp.NonTerminal):
                next_action: A = tabular_ega(
                    q=q,
                    states=next_state,
                    actions=set(mdp.actions(next_state)),
                    epsilon=epsilon
                )
                q[state][action] += (1/counts[(state,action)])*\
                    (reward + gamma*q[next_state][next_action] - q[state][action])
                action = next_action
            else:
                q[state][action] += (1/counts[(state,action)])*reward
            yield q
            steps += 1
            state = next_state